In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

def read_arff(filepath):
    data = arff.loadarff(filepath)
    df = pd.DataFrame(data[0], dtype=np.float32)
    df.to_csv(filepath.replace('.arff','.dat'),index=None, columns=None, header=None)

In [3]:
read_arff('data/benchmark/realWorld/elecNormNew.arff')

In [4]:
data = pd.read_csv('data/benchmark/realWorld/elecNormNew.dat', header=None)
# data['cls'] = data.iloc[:,-1].map(lambda x: 0 if x == "b'UP'" else 1)
data.iloc[:,-1] = data.iloc[:,-1].map(lambda x: 0 if x == "b'UP'" else 1)

df = pd.DataFrame(data.values)
df.to_csv('data/benchmark/elecNormNew.csv', header=None, columns=None, index=None)
# np.save('data/elecNormNew.npy', data.values)

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

def mm_norm(src, des, sep=',', header=False):
    if header:
        df = pd.read_csv(src, sep=sep)
    else:
        df = pd.read_csv(src,header=None, sep=sep)
        
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    df_mm = MinMaxScaler(feature_range=[0,2]).fit_transform(X)
#     newdata = np.concatenate([df_mm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    df_mm_norm = Normalizer().fit_transform(df_mm)
    newdata = np.concatenate([df_mm_norm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    df = pd.DataFrame(newdata)
    df.describe()
    df.to_csv(des, index=None, columns=None, header=None)

# HAR

In [56]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

path = 'data/benchmark/realWorld/HAR'

file_x = os.path.join(path,'X_train.txt')
file_y = os.path.join(path,'y_train.txt')
file_x_new = file_x.replace('.txt','.dat')
des = 'data/data/har.csv'

def har():
    def trans(file_x):
        with open(file_x) as fin:
            with open(file_x_new,'w') as fout:
                for line in fin.readlines():
                    new_line = line.strip().replace('  ',' ').replace(' ',',')
                    fout.write(f'{new_line}\n')
#     trans(file_x)

    df_x = pd.read_csv(file_x_new, header=None)
    df_y = pd.read_csv(file_y, header=None)
    
    data_x = df_x.values
    data_y = df_y.values
    
    df_mm = MinMaxScaler(feature_range=[0,2]).fit_transform(data_x)
#     newdata = np.concatenate([df_mm, data_y], axis=1)

    df_mm_norm = Normalizer().fit_transform(df_mm)
    newdata = np.concatenate([df_mm_norm, data_y], axis=1)

    df = pd.DataFrame(newdata)
    df.to_csv(des, index=None, columns=None, header=None)

har()

# Forest Covertype 

In [63]:
src = 'data/dataset/covtype.data'
des = 'data/data/covtype.csv'

mm_norm(src, des)

# Gas Sensor

In [5]:
src = 'data/dataset/gas-sensor.dat'
des = 'data/data/gas-sensor.csv'

mm_norm(src, des)

# kddcup

In [70]:
import csv
import time
from sklearn.model_selection import train_test_split

global label_list
label_list = []

dat = 'data/dataset/kddcup.data_10_percent_corrected'
src = 'data/dataset/kddcup.data_10_percent_corrected.dat'
des = 'data/data/kddcup.data_10_percent_corrected.csv'

def preprocess(src, des):
    data_file = open(des, 'w', newline='')
    with open(src) as data_source:
        csv_reader = csv.reader(data_source)
        csv_writer = csv.writer(data_file)
        count = 0
        for i, row in enumerate(csv_reader):
            temp_line = np.array(row)
            temp_line[1] = handleProtocol(row)
            temp_line[2] = handleService(row)
            temp_line[3] = handleFlag(row)
            temp_line[41] = handleLabel(row)
            csv_writer.writerow(temp_line)
            count += 1
#             if count % 100000 == 0:
#                 print(temp_line)
    data_file.close()

def handleProtocol(input):
    """
    定义将源文件行中3种协议类型转换成数字标识的函数
    """
    protocol_list = ['tcp', 'udp', 'icmp']
    if input[1] in protocol_list:
        return protocol_list.index(input[1])

def handleService(input):
    """
    定义将源文件行中70种网络服务类型转换成数字标识的函数
    """
    service_list = ['aol', 'auth', 'bgp', 'courier', 'csnet_ns', 'ctf', 'daytime', 'discard', 'domain', 'domain_u',
                    'echo', 'eco_i', 'ecr_i', 'efs', 'exec', 'finger', 'ftp', 'ftp_data', 'gopher', 'harvest',
                    'hostnames',
                    'http', 'http_2784', 'http_443', 'http_8001', 'imap4', 'IRC', 'iso_tsap', 'klogin', 'kshell',
                    'ldap',
                    'link', 'login', 'mtp', 'name', 'netbios_dgm', 'netbios_ns', 'netbios_ssn', 'netstat', 'nnsp',
                    'nntp',
                    'ntp_u', 'other', 'pm_dump', 'pop_2', 'pop_3', 'printer', 'private', 'red_i', 'remote_job', 'rje',
                    'shell',
                    'smtp', 'sql_net', 'ssh', 'sunrpc', 'supdup', 'systat', 'telnet', 'tftp_u', 'tim_i', 'time',
                    'urh_i', 'urp_i',
                    'uucp', 'uucp_path', 'vmnet', 'whois', 'X11', 'Z39_50']
    if input[2] in service_list:
        return service_list.index(input[2])

def handleFlag(input):
    """
    定义将源文件行中11种网络连接状态转换成数字标识的函数
    """
    flag_list = ['OTH', 'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2', 'S3', 'SF', 'SH']
    if input[3] in flag_list:
        return flag_list.index(input[3])

def handleLabel(input):
    """
    定义将源文件行中攻击类型转换成数字标识的函数(训练集中共出现了22个攻击类型，而剩下的17种只在测试集中出现)
    """
    global label_list
    if not input[41] in label_list:
        label_list.append(input[41])
    return label_list.index(input[41])


def kddcup99():
    preprocess(dat, src)
    mm_norm(src, des)

kddcup99()

['0' '2' '12' '9' '1032' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '511' '511' '0.00' '0.00' '0.00' '0.00' '1.00' '0.00'
 '0.00' '255' '255' '1.00' '0.00' '1.00' '0.00' '0.00' '0.00' '0.00'
 '0.00' '5']
['0' '2' '12' '9' '1032' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '509' '509' '0.00' '0.00' '0.00' '0.00' '1.00' '0.00'
 '0.00' '255' '255' '1.00' '0.00' '1.00' '0.00' '0.00' '0.00' '0.00'
 '0.00' '5']
['0' '2' '12' '9' '1032' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '511' '511' '0.00' '0.00' '0.00' '0.00' '1.00' '0.00'
 '0.00' '255' '255' '1.00' '0.00' '1.00' '0.00' '0.00' '0.00' '0.00'
 '0.00' '5']
['0' '2' '12' '9' '520' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '478' '478' '0.00' '0.00' '0.00' '0.00' '1.00' '0.00'
 '0.00' '255' '255' '1.00' '0.00' '1.00' '0.00' '0.00' '0.00' '0.00'
 '0.00' '5']


# MNIST

# occupancy

In [75]:
dat = 'data/dataset/occupancy.txt'
src = 'data/dataset/occupancy.dat'
des = 'data/data/occupancy.txt'

def occupancy():
    def trans():
        df = pd.read_csv(dat)
        df = df.iloc[:,2:]
        df.to_csv(src, index=None, columns=None, header=None)
    
    trans()
    mm_norm(src, des)

occupancy()

# poker

In [79]:
dat = 'data/dataset/poker-lsn.arff'
src = 'data/dataset/poker-lsn.dat'
des = 'data/data/poker-lsn.csv'

# read_arff(dat)
mm_norm(src, des)

# sensorless-drive

In [80]:
src = 'data/dataset/Sensorless_drive_diagnosis.txt'
des = 'data/data/sensorless.csv'

mm_norm(src, des, sep=' ')

# shuttle

In [81]:
src = 'data/dataset/shuttle.trn'
des = 'data/data/shuttle.csv'

mm_norm(src, des, sep=' ')

# spam

In [89]:
dat = 'data/dataset/spam_data.arff'
src = 'data/dataset/spam_data.dat'
des = 'data/dataset/spam.csv'

def spam():
    def trans():
        data = arff.loadarff(dat)
        df = pd.DataFrame(data[0])
        # # 样本
        # 对标签进行处理
        sample = df.values[:, :-1]
        label = df.values[:, -1]
        cla = []
        dic = {
            b'spam': 0,
            b'legitimate': 1
        }
        for i in label:
            cla.append(dic[i])
        data = np.concatenate([sample, np.array(cla).reshape([-1, 1])], axis=1).astype(np.int8)
        df = pd.DataFrame(data)
        df.to_csv(src, index=None, columns=None, header=None)
    trans()
    mm_norm(src, des)
spam()

   href   our  font  size color please align remove  face bgcolor  ...  \
0  b'0'  b'0'  b'1'  b'1'  b'1'   b'1'  b'0'   b'0'  b'1'    b'0'  ...   
1  b'1'  b'1'  b'1'  b'1'  b'1'   b'1'  b'1'   b'0'  b'1'    b'0'  ...   
2  b'0'  b'0'  b'0'  b'0'  b'0'   b'0'  b'0'   b'0'  b'0'    b'0'  ...   
3  b'1'  b'1'  b'1'  b'1'  b'1'   b'1'  b'0'   b'0'  b'1'    b'0'  ...   
4  b'0'  b'0'  b'0'  b'0'  b'0'   b'0'  b'0'   b'1'  b'0'    b'0'  ...   

  receipt websites removeme 3dname 22http   555 promotions   101   was  \
0    b'0'     b'0'     b'0'   b'0'   b'0'  b'0'       b'0'  b'0'  b'0'   
1    b'0'     b'0'     b'0'   b'1'   b'0'  b'0'       b'0'  b'0'  b'0'   
2    b'0'     b'0'     b'0'   b'0'   b'0'  b'0'       b'0'  b'0'  b'0'   
3    b'0'     b'0'     b'0'   b'0'   b'0'  b'0'       b'0'  b'0'  b'1'   
4    b'0'     b'0'     b'0'   b'0'   b'0'  b'0'       b'0'  b'0'  b'0'   

  spamorlegitimate  
0          b'spam'  
1          b'spam'  
2          b'spam'  
3          b'spam'  
4    

In [ ]:
def spam():
    from scipy.io import arff
    import pandas as pd

    filepath = 'data/spam/spam_data.arff'

    data = arff.loadarff(filepath)
    df = pd.DataFrame(data[0])
    # print(df.describe())
    # print(df.head())
    # # 样本
    # 对标签进行处理
    sample = df.values[:, :-1]
    label = df.values[:, -1]
    cla = []
    dic = {
        b'spam': 0,
        b'legitimate': 1
    }
    for i in label:
        cla.append(dic[i])
    data = np.concatenate([sample, np.array(cla).reshape([-1, 1])], axis=1).astype(np.int8)
    # np.save('data/spam/spam.array', data)
    df = pd.DataFrame(data)
    df.to_csv('data/spam/spam.csv', index=None, columns=None, header=None)

In [11]:
from scipy.io import arff
import pandas as pd
import numpy as np

filepath = 'data/poker-lsn.arff'

data = arff.loadarff(filepath)
df = pd.DataFrame(data[0], dtype=np.float32)
print(df.describe())
print(df.head())de
# df = pd.DataFrame(df.values)
# df.to_csv('data/poker-lsm.csv', index=None, columns=None, header=None)

# # 样本
# 对标签进行处理
# sample = df.values[:, :-1]
# label = df.values[:, -1]
# cla = []
# dic = {
#     b'spam': 0,
#     b'legitimate': 1
# }
# for i in label:
#     cla.append(dic[i])
# data = np.concatenate([sample, np.array(cla).reshape([-1, 1])], axis=1).astype(np.int8)

                  s1             r1             s2             r2  \
count  829201.000000  829201.000000  829201.000000  829201.000000   
mean        2.376337       2.613158       2.506348       4.788464   
std         1.113204       1.747380       1.116415       2.241039   
min         1.000000       1.000000       1.000000       1.000000   
25%         1.000000       1.000000       2.000000       3.000000   
50%         2.000000       2.000000       3.000000       5.000000   
75%         3.000000       4.000000       4.000000       6.000000   
max         4.000000      12.000000       4.000000      13.000000   

                  s3             r3             s4             r4  \
count  829201.000000  829201.000000  829201.000000  829201.000000   
mean        2.499307       6.999647       2.493204       9.210980   
std         1.117813       2.374738       1.116072       2.239432   
min         1.000000       1.000000       1.000000       1.000000   
25%         1.000000       5.0000

In [ ]:
# 同一数据格式
import os

import pandas as pd

task = 'occupancy'

def mm_std(path, des, sep):
    df = pd.read_csv(path,header=None, sep=sep)
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    df_mm = MinMaxScaler().fit_transform(X)
    df_mm = pd.DataFrame(df_mm)
    print('df_mm:\n', df_mm.describe())
    newdata = np.concatenate([df_mm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    # df_mm_std = StandardScaler().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())

    # df_mm_std = Normalizer().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())
    # newdata = np.concatenate([df_mm_std, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    df = pd.DataFrame(newdata)
    df.to_csv(des, index=None, columns=None, header=None)

# iris数据
def iris(path='data/iris/raw/iris.data'):
    data = pd.read_csv(path, sep=',', header=None)
    dic = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
    columns = data.columns
    data[columns[-1]] = data[columns[-1]].map(dic)
    data.to_csv(path.replace('.data', '.dat'), header=None, index=False, sep=' ')


# iris(path)


# weather 数据集
def weather():
    path = 'data/weather/weather.mat'
    from scipy.io import loadmat
    import numpy
    annots = loadmat(path)
    data = annots['weather']
    numpy.savetxt(path.replace('.mat', '.tsv'), data, delimiter=',')

    mm_std(path.replace('.mat','.tsv'),path.replace('.mat','.dsv'),sep=',')

# shuttle
def shuttle():
    # path = 'data/shuttle/shuttle_Norm.trn'
    # from scipy.io import loadmat
    # import numpy
    # annots = loadmat(path)
    # data = annots['shuttle_Norm']
    # numpy.savetxt(path.replace('.mat', '.csv'), data, delimiter=',')
    src = 'data/shuttle/shuttle.trn'
    des = 'data/shuttle/shuttle.csv'
    mm_std(src, des, sep=' ')

# spam 数据集
def spam():
    from scipy.io import arff
    import pandas as pd

    filepath = 'data/spam/spam_data.arff'

    data = arff.loadarff(filepath)
    df = pd.DataFrame(data[0])
    # print(df.describe())
    # print(df.head())
    # # 样本
    # 对标签进行处理
    sample = df.values[:, :-1]
    label = df.values[:, -1]
    cla = []
    dic = {
        b'spam': 0,
        b'legitimate': 1
    }
    for i in label:
        cla.append(dic[i])
    data = np.concatenate([sample, np.array(cla).reshape([-1, 1])], axis=1).astype(np.int8)
    # np.save('data/spam/spam.array', data)
    df = pd.DataFrame(data)
    df.to_csv('data/spam/spam.csv', index=None, columns=None, header=None)

# UCI
def uci():
    with open('data/UCI HAR Dataset/train/X_train.txt','r') as f:
        length = []
        for line in f.readlines():
            l = line.strip().split(' ')
            if len(l) not in length:
                length.append(len(l))
        print(length)

    X = pd.read_csv('data/UCI HAR Dataset/train/X_train.txt', header=None, sep=' ')
    print(X)
    X = X.values
    print(X)
    y = pd.read_csv('data/UCI HAR Dataset/train/y_train.txt', header=None, sep=',').values
    df_mm = MinMaxScaler().fit_transform(X)
    df_mm = pd.DataFrame(df_mm * 100)
    print('df_mm:\n', df_mm.describe())
    newdata = np.concatenate([df_mm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)
    df = pd.DataFrame(newdata)
    df.to_csv(des, index=None, columns=None, header=None)

# GSD
def gsd():
    # for i in range(1,11):
    #     file = f'data/GSD/batch{i}.dat'
    #     with open(file.replace('.dat','.csv'),'w') as fout:
    #         with open(file) as f:
    #             for line in f.readlines():
    #                 line = line.strip().split(' ')
    #                 t = []
    #                 for j in line[1:]:
    #                     t.append(j.split(':')[1]+',')
    #                 s = ''.join(t)
    #                 fout.write(s+line[0]+'\n')
    # for i in range(1,11):
    #     file = f'data/GSD/batch{i}.csv'
    #     mm_std(file,file.replace('.csv','.tsv'),',')
    with open('data/GSD/gsd.csv', 'w') as fout:
        lines = []
        for i in range(1,11):
            with open(f'data/GSD/batch{i}.tsv') as f:
                lines.extend(f.readlines())
        fout.writelines(lines)
# kddcup99
import numpy as np
import pandas as pd
import csv
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split

global label_list
label_list = []

src_file = 'data/kddcup99/kddcup.data_10_percent_corrected'
des_file = 'data/kddcup99/kddcup.data_10_percent_corrected.csv'

def preprocess(src, des):
    data_file = open(des, 'w', newline='')
    with open(src) as data_source:
        csv_reader = csv.reader(data_source)
        csv_writer = csv.writer(data_file)
        count = 0
        for i, row in enumerate(csv_reader):
            temp_line = np.array(row)
            temp_line[1] = handleProtocol(row)
            temp_line[2] = handleService(row)
            temp_line[3] = handleFlag(row)
            temp_line[41] = handleLabel(row)
            csv_writer.writerow(temp_line)
            count += 1
            if count % 100000 == 0:
                print(temp_line)
    data_file.close()


def handleProtocol(input):
    """
    定义将源文件行中3种协议类型转换成数字标识的函数
    """
    protocol_list = ['tcp', 'udp', 'icmp']
    if input[1] in protocol_list:
        return protocol_list.index(input[1])


def handleService(input):
    """
    定义将源文件行中70种网络服务类型转换成数字标识的函数
    """
    service_list = ['aol', 'auth', 'bgp', 'courier', 'csnet_ns', 'ctf', 'daytime', 'discard', 'domain', 'domain_u',
                    'echo', 'eco_i', 'ecr_i', 'efs', 'exec', 'finger', 'ftp', 'ftp_data', 'gopher', 'harvest',
                    'hostnames',
                    'http', 'http_2784', 'http_443', 'http_8001', 'imap4', 'IRC', 'iso_tsap', 'klogin', 'kshell',
                    'ldap',
                    'link', 'login', 'mtp', 'name', 'netbios_dgm', 'netbios_ns', 'netbios_ssn', 'netstat', 'nnsp',
                    'nntp',
                    'ntp_u', 'other', 'pm_dump', 'pop_2', 'pop_3', 'printer', 'private', 'red_i', 'remote_job', 'rje',
                    'shell',
                    'smtp', 'sql_net', 'ssh', 'sunrpc', 'supdup', 'systat', 'telnet', 'tftp_u', 'tim_i', 'time',
                    'urh_i', 'urp_i',
                    'uucp', 'uucp_path', 'vmnet', 'whois', 'X11', 'Z39_50']
    if input[2] in service_list:
        return service_list.index(input[2])


def handleFlag(input):
    """
    定义将源文件行中11种网络连接状态转换成数字标识的函数
    """
    flag_list = ['OTH', 'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2', 'S3', 'SF', 'SH']
    if input[3] in flag_list:
        return flag_list.index(input[3])


def handleLabel(input):
    """
    定义将源文件行中攻击类型转换成数字标识的函数(训练集中共出现了22个攻击类型，而剩下的17种只在测试集中出现)
    """
    global label_list
    if not input[41] in label_list:
        label_list.append(input[41])
    return label_list.index(input[41])


def kddcup99():
    preprocess(src_file, des_file)

    df = pd.read_csv(des_file, header=None)

    df_mm = MinMaxScaler().fit_transform(df)
    df_mm = pd.DataFrame(df_mm)
    print('df_mm:\n', df_mm.describe())

    df_mm_std = StandardScaler().fit_transform(df_mm.iloc[:, :-1])
    df_mm_std = pd.DataFrame(df_mm_std)
    print('df_mm_std:\n', df_mm_std.describe())

    newdata = np.concatenate([df_mm_std, np.array(df.iloc[:, -1], dtype=np.int8).reshape([-1, 1])], axis=1)
    df = pd.DataFrame(newdata)
    df.to_csv('kddcup99.data.mm_std', index=None, columns=None, header=None)

# sensor task
def sensor():
    src = 'data/sensor/sensor.txt'
    des = 'data/sensor/sensor.csv'
    mm_std(src, des, ' ')

def covtyep():
    src = 'data/covtype/covtype.data'
    des = 'data/covtype/covtype.csv'
    mm_std(src,des,sep=',')

def poker():
    src = 'data/poker-hand/poker-hand-training-true.data'
    des = 'data/poker-hand/poker-hand.csv'
    mm_std(src,des,sep=',')

def occupancy():
    src = 'data/occupancy_data/datatraining.txt'
    des = 'data/occupancy_data/datatraining.csv'
    mm_std(src, des, sep=',')

def electricity():
    src = 'data/ElectricityLoadDiagrams20112014/LD2011_2014.txt'
    des = 'data/ElectricityLoadDiagrams20112014/LD2011_2014.csv'
    data = pd.read_csv(src, sep=';')
    df = data.iloc[:,1:]

    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    df_mm = MinMaxScaler().fit_transform(X)
    df_mm = pd.DataFrame(df_mm)
    print('df_mm:\n', df_mm.describe())
    newdata = np.concatenate([df_mm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    # df_mm_std = StandardScaler().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())

    # df_mm_std = Normalizer().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())
    # newdata = np.concatenate([df_mm_std, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    df = pd.DataFrame(newdata)
    df = df.drop(columns=[2])
    print(df)
    df.to_csv(des, index=None, columns=None, header=None)

def gas():
    # file_names = os.listdir('data/gas-sensor')
    # data = pd.read_csv(os.path.join(f'data/gas-sensor/{file_names[0]}'), sep=' ')
    # data = data.values
    # for file in file_names[1:]:
    #     path = os.path.join('data/gas-sensor', file)
    #     temp = pd.read_csv(path, sep=' ')
    #     temp = temp.values
    #     data = np.concatenate([data, temp])
    # print(data.shape)
    #
    # data = pd.DataFrame(data)
    # for i,col in enumerate(data.columns):
    #    print(i, col)
    # for i,col in enumerate(data.columns):
    #     if i == 0:
    #         continue
    #     else:
    #         data[col] = data[col].apply(lambda x:x.split(':')[1])
    # data = np.concatenate([data.iloc[:,1:], np.array(data.iloc[:,0], dtype=np.int8).reshape([-1, 1])], axis=1)
    # data = pd.DataFrame(data)
    # data.to_csv('data/gas-sensor/data.dat', header=None, index=None)
    src = 'data/gas-sensor/data.dat'
    des = 'data/gas-sensor/data.csv'
    mm_std(src,des,sep=',')

def har():
    src = 'data/HAR/train/X_train.txt'
    with open('data/HAR/train/X_train.dat', 'w') as fout:
        with open(src) as f:
            for line in f.readlines():
                newline = line.strip()
                fout.write(newline)

    des = 'data/HAR/x_train.csv'
    X = pd.read_csv(src.replace('.txt','.dat'), sep=' ')
    y = pd.read_csv(src.replace('X_train','y_train'), header=None).values
    df_mm = MinMaxScaler().fit_transform(X)
    df_mm = pd.DataFrame(df_mm)
    print('df_mm:\n', df_mm.describe())
    newdata = np.concatenate([df_mm, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    # df_mm_std = StandardScaler().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())

    # df_mm_std = Normalizer().fit_transform(df_mm)
    # df_mm_std = pd.DataFrame(df_mm_std)
    # print('df_mm_std:\n', df_mm_std.describe())
    # newdata = np.concatenate([df_mm_std, np.array(y, dtype=np.int8).reshape([-1, 1])], axis=1)

    df = pd.DataFrame(newdata)
    df.to_csv(des, index=None, columns=None, header=None)

# *********************************************************************************************
def dataPrepare():
    # 读取正常数据，异常数据并合并

    # 读取正常数据 使用数据 Danmini_Doorbell 物理设备。
    data = pd.read_csv("data/BIot/Danmini_Doorbell/benign_traffic.csv")
    data['Class'] = 0

    # 读取异常数据mirai_attacks
    udp = pd.read_csv("data/BIot/Danmini_Doorbell/mirai_attacks/udp.csv")
    udp['Class'] = 1

    ack = pd.read_csv("data/BIot/Danmini_Doorbell/mirai_attacks/ack.csv")
    ack['Class'] = 1

    scan = pd.read_csv("data/BIot/Danmini_Doorbell/mirai_attacks/scan.csv")
    scan['Class'] = 1

    syn = pd.read_csv("data/BIot/Danmini_Doorbell/mirai_attacks/syn.csv")
    syn['Class'] = 1

    udpplain = pd.read_csv("data/BIot/Danmini_Doorbell/mirai_attacks/udpplain.csv")
    udpplain['Class'] = 1

    # 读取异常数据gafgyt_attacks
    g_combo = pd.read_csv("data/BIot/Danmini_Doorbell/gafgyt_attacks/combo.csv")
    g_combo['Class'] = 1

    g_junk = pd.read_csv("data/BIot/Danmini_Doorbell/gafgyt_attacks/junk.csv")
    g_junk['Class'] = 1

    g_scan = pd.read_csv("data/BIot/Danmini_Doorbell/gafgyt_attacks/scan.csv")
    g_scan['Class'] = 1

    g_tcp = pd.read_csv("data/BIot/Danmini_Doorbell/gafgyt_attacks/tcp.csv")
    g_tcp['Class'] = 1

    g_udp = pd.read_csv("data/BIot/Danmini_Doorbell/gafgyt_attacks/udp.csv")
    g_udp['Class'] = 1

    frames = [data, udp, ack, scan, syn, udpplain
        , g_combo, g_junk, g_scan, g_tcp, g_udp]

    # 结果拼接
    result = pd.concat(frames, ignore_index=True)

    result.to_csv('danmini_doorbell.csv')
    return result
# *********************************************************************************************
if task == 'kddcup99':
    kddcup99()
    src = 'data/kddcup99/kddcup.data_10_percent_corrected.csv'
    des = 'data/kddcup99/kddcup.data.tsv'
    mm_std(src,des,',')
elif task == 'iris':
    iris()
elif task == 'weather':
    weather()
elif task == 'spam':
    spam()
elif task == 'sensor':
    sensor()
elif task == 'shuttle':
    shuttle()
elif task == 'UCI':
    uci()
elif task == 'GSD':
    gsd()
elif task == 'covtype':
    covtyep()
elif task == 'electricity':
    electricity()
elif task == 'gas':
    gas()
elif task == 'occupancy':
    occupancy()
elif task == 'poker':
    poker()
elif task == 'HAR':
    har()